# Claims Processing
this code opens the large downloaded claims file, opens the list of patents numbers and the outcome variable and loops though adding independent claim text and dependent claim count

This is really slow and should be sped up, but needs some rewriting to fix this.
Also currently I only save a subset of the information, so that could be improved.
If I do want to parralellize it, I need to keep in mind how to store order (for dependent claims)
Also be conscious of type errors - patent numbers being strings or integers and checking equality

Also, since I am somewhat randomly searching the file

Note that prior to doing this, I split the file into different files with 10000 lines using 
`split -l 1000000 claim.tsv`


Striped directory `lfs setstripe --count 20 split_claims`

In [ ]:
# when kernel reconnects, make sure code is up to date
# save small claims files instead of one big one?
# update description with correct file paths
# run claims code on engaging
# move description and reference files to correct directory
# run description and referenceto make sure they work properly with new sample
# make sure a check and replace option for processing
# rerun with just numerical reference counts

In [1]:
import os
import csv
import concurrent
import pandas as pd
import numpy as np
import gzip
import pickle

import funcy

In [2]:
csv.field_size_limit(100000000)
# need to do this because claims file loop kept stopping because of the error due to fields longer than max field size
# I deal with really long fields by only taking the first 2000 characters or so
csv.field_size_limit()

100000000

In [3]:
# Split description directory
index_file = '/pool001/lraymond/patent_data/index_files/pat_nums_index.csv'
TO_SAVE_INDEX_FILE = '/pool001/lraymond/patent_data/index_files/pat_nums_plus_claims.gzip'
CLAIMS_FILES_DIR = '/nobackup1/lraymond/patent_data/split_claims'
TO_SAVE_CLAIMS_DIR = '/pool001/lraymond/processed_data/claims'
# I should be story these files in the lustre parallel file system
# '/nobackup1/lraymond/patent_data/split_brief_description'
# I want to stripe this directory over 20 servers to optimize performance 
# lfs setstripe --count 20 patent_data/split_brief_description
# old_dir on laptop = '/Users/Lraymond/Python/patents/data/patent_api_data/split_test'

In [4]:
# pats_index = load_zipped_pickle(index_file)
pats_index = pd.read_csv(index_file)

pats_index = pats_index.sort_values('patent_number')

/home/software/sloan/local/lib/py36/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def save_zipped_pickle(obj, filename, protocol=-1):
    # this will default to the highest protocol
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, protocol)


def load_zipped_pickle(filename):
    # load a zipped compressed pickle file
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object


In [6]:
# save the patent text in a dictionary
def create_claims_dicts(patent_nums):
    # create independnet and dependent claims dictionaries
    return dict(zip(patent_nums, funcy.repeat(None)))

def create_claim_dataframe(patent_nums):
    nums = pd.Series(patent_nums).astype(int).drop_duplicates().copy()
    patent_random_sample = pd.DataFrame(0, 
        index=nums, columns=['max_independent_claim',
                             'number_independent_claims_found',
                             'number_dependent_claims_found'])
    return patent_random_sample

In [7]:
def save_independent_claims(patent_num, txt, seq, to_save_dir=TO_SAVE_CLAIMS_DIR):
    fname = os.path.join(to_save_dir, 'claims_{}.gzip'.format(patent_num))
    if os.path.exists(fname):
        # append claim to 
        existing = load_zipped_pickle(fname)
        existing.append((txt, seq))
        e2 = sorted(existing, key=lambda x: x[1])
    else:
        e2 = [(txt, seq)]
    save_zipped_pickle(e2, fname)

In [8]:
def process_independent_claim(patent_num, txt, seq, text_claim_dict, numerical_claim_df):
    # save text to the text dictionary
    # save independent and dependent claim text to the text dictionaries
#     if text_claim_dict[patent_num] is None:
#         text_claim_dict[patent_num] = [(txt, seq),]
#     else:
#         text_claim_dict[patent_num].append((txt, seq))
    save_independent_claims(patent_num, txt, seq)
    # update dataframe with max sequence of independent claim and number found
    existing = numerical_claim_df.loc[patent_num, 'max_independent_claim']
    numerical_claim_df.loc[patent_num, 'max_independent_claim'] = max(existing, seq)
    numerical_claim_df.loc[patent_num, 'number_independent_claims_found'] += 1
    return None

def process_dependent_claim(patent_num, txt, seq, claim_dependence, text_claim_dict, numerical_claim_df):
    # save text to the text dictionary
    # save independent and dependent claim text to the text dictionaries
    if text_claim_dict[patent_num] is None:
        text_claim_dict[patent_num] = [(txt, seq, claim_dependence),]
    else:
        text_claim_dict[patent_num].append((txt, seq, claim_dependence))
        # because all depednent claims have a NULL field for claim dependence, not putting a lot of stock in them
    # update dataframe with max sequence of independent claim and number found
#     existing = numerical_claim_df.loc[patent_num, 'max_dependent_claim']
#     numerical_claim_df.loc[patent_num, 'max_dependent_claim'] = max(existing, seq)
    numerical_claim_df.loc[patent_num, 'number_dependent_claims_found'] += 1
    return None


def check_get_independent_claim(claim_dependence_str):
    # if claim is independent, this should be -1, but value comes in as a string and str.isdigit() returns False for -1
    if claim_dependence_str is None or len(claim_dependence_str) == 0:
        return False, None
    if claim_dependence_str == '-1':
        return True, -1
    valid_int = claim_dependence_str.isdigit()
    if valid_int:
        return False, int(claim_dependence_str)
    return False, None
        # this is an independent claim

## Known Issues with Claims Files
* the claim dependence field is "Null" for a bunch of claims
* I am not including these in either the independent or dependent claim field at the moment
* there are only a subset of numbers where this is an issue- they all have "Null" in the independent claim field and the claim text is as follows
`4785101	3,4-Dihydro-6-[5-(3-hydroxy-pyrazolyl)]-2(1H)-quinazolinone or a pharmaceuticlaly acceptable salt thereof.	NULL	18	False`
* I will mark these as dependent and process them as that for now
* patent numbers with these issues are 
`{4581354, 4658060, 4659762, 4748238, 4785101, 4797495, 4879313, 4996207,  5053511, 5294612}`

In [9]:
def yield_file_lines(
    filename, list_patent_nums, valid_row_len, max_lines=None, patent_number_index=1):
    # create iterator to process file
    # create set to check membership
    # only returns lines with patent numbers in the set and those with a valid number of fields
    # row is returned as a list with a length equal to the number of fields available
    set_patent_nums = set(list_patent_nums)
    with open(filename) as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t', quoting=csv.QUOTE_NONE)
        for row in tsvin:
            if max_lines and tsvin.line_num > max_lines:
                # if a max line limit, break out of the generator
                return
            else:
                # only process if the row has all fields
                if len(row) == valid_row_len:
                    patent_num_field = row[patent_number_index]
                    is_num = patent_num_field.isdigit()

                    if is_num and int(patent_num_field) in set_patent_nums:
                        # only yield relevant lines
                        yield row

In [10]:
def process_claims_information(claims_filename, patent_nums_list,
                               max_lines=None, claim_files_dir=CLAIMS_FILES_DIR):
    # ind_dict = create_claims_dicts(patent_nums_list)
    # just processing and saving claims, 
    ind_dict = {}

    claim_numerical_df = create_claim_dataframe(patent_nums_list)
    claims_fname = os.path.join(claim_files_dir, claims_filename)
    try:
        for line in yield_file_lines(
            claims_fname, patent_nums_list, 6, max_lines, patent_number_index=1):
            # claim dependence is sequence number of claim this is dependent on. -1 if independent
            # sequence is order in which claims appear in patent file
            # exemplary, the last field is whether the claim is one of the exemplary claims of the patent
            # these appear to be claims that include the words "for example" but not totally sure about this
            
            _, str_pat_num, txt, claim_dependence_str, seq_str, _ = line
            patent_num = int(str_pat_num)
            # something is going wrong with this line
            is_independent, claim_dependence = check_get_independent_claim(claim_dependence_str)   
            seq = int(seq_str) if seq_str.isdigit() else None
            # claim dependence is -1 for independent claims
            if is_independent:
                process_independent_claim(patent_num, txt, seq, ind_dict, claim_numerical_df)
                # for now, I am going to skip processing dependent claims
#             else:
#                 # this includes cases when seq is nan
#                 process_dependent_claim(patent_num, txt, seq, claim_dependence, dep_dict, claim_numerical_df)
    except (Exception, KeyboardInterrupt) as e:
        print(e)
    finally:
        # always execute this piece of code even after an exception
        return claim_numerical_df, _

In [22]:
# get list of patent numbers to scrape
patent_nums_list = pats_index.patent_number.values
txt_files = os.listdir(CLAIMS_FILES_DIR)
print(txt_files, patent_nums_list.shape)


['xak', 'xcn', 'xbd', 'xck', 'xcl', 'xdq', 'xbu', 'xbl', 'xbf', 'xbs', 'xdj', 'xbz', 'xar', 'xbk', 'xai', 'xdm', 'xam', 'xcp', 'xde', 'xcz', 'xal', 'xcg', 'xcf', 'xcr', 'xcm', 'xbb', 'xbe', 'xaa', 'xaf', 'xas', 'xcs', 'xac', 'xcw', 'xbi', 'xdd', 'xbv', 'xaz', 'xdb', 'xbo', 'xbg', 'xbw', 'xdh', 'xay', 'xba', 'xbt', 'xby', 'xbj', 'xab', 'xbh', 'xag', 'xaq', 'xao', 'xca', 'xco', 'xdc', 'xdl', 'xdi', 'xbq', 'xbx', 'xav', 'xaj', 'xcu', 'xah', 'xcj', 'xan', 'xcy', 'xae', 'xdf', 'xcb', 'xau', 'xct', 'xaw', 'xax', 'xdp', 'xbp', 'xdk', 'xat', 'xbm', 'xce', 'xcd', 'xbc', 'xdg', 'xap', 'xcx', 'xda', 'xcv', 'xdn', 'xbr', 'xci', 'xad', 'xch', 'xbn', 'xdo', 'xcq', 'xcc'] (1994958,)


In [ ]:
# in returned item, first is a dataframe
dict_results = dict(zip(txt_files, map(
    lambda x: process_claims_information(x, patent_nums_list), txt_files)))

In [ ]:
def concat_dfs(dict_results):
    # combine dataframes by adding a column with filename source
    # then reset index so patent number is a column, append all df together
    # then summarize by apply max/sum functions to respective  columns
    # process dataframe associated with the results
    list_df = []
    for fname, res in dict_results.items():
        # get dataframe
        mini_df = res[0]
        print(fname)
        mini_df['source_filename'] = fname
        mini_df = mini_df.reset_index(drop=False)
        mini_df = mini_df.rename(columns={'index': 'patent_number'})
        list_df.append(mini_df)
    # memory concerns so deleting dict results
    del dict_results
    return pd.concat(list_df, axis=0, join='outer', ignore_index=True)


def find_filenames_with_info(patent_num, ungrouped_df):
    filenames = ungrouped_df.loc[
        ((ungrouped_df.patent_number==patent_num) & (
            ungrouped_df['number_independent_claims_found'] > 0)), 'source_filename'].values
    if len(filenames) > 0:
        return ' ; '.join(filenames)
    return np.nan

In [ ]:
master_df = concat_dfs(dict_results)

In [ ]:
master_df2 = master_df.sort_values(['patent_number', 'source_filename']).groupby('patent_number').agg(
    {'max_independent_claim': np.max, 
     'number_independent_claims_found': np.sum,
    'number_dependent_claims_found': np.sum})

In [ ]:
master_df2['independent_claims_filenames'] = np.nan
filled_vals_mask = (master_df2.number_independent_claims_found > 0) 

master_df2.loc[filled_vals_mask, 'independent_claims_filenames'] = master_df2.loc[
    filled_vals_mask,:].index.map(
        lambda x: find_filenames_with_info(x, master_df))

In [ ]:

master_df3 = master_df2.reset_index(drop=False).drop_duplicates('patent_number')

print(master_df.shape)
print(master_df2.shape)
print(master_df3.shape)

In [ ]:
del master_df
del master_df2

In [ ]:
pats2 = pats_index.sort_values(
    'patent_number').merge(master_df3, on='patent_number', how='left')

In [ ]:
pats2[
    'flag_has_independent_claims'] = pats2.number_independent_claims_found.apply(
        lambda x: int(x>0))

In [ ]:
print(pats2.shape)
print(pats2.drop_duplicates('patent_number').shape)

In [ ]:

save_zipped_pickle(pats2, TO_SAVE_INDEX_FILE) 